In [13]:
import os
import jieba
from sklearn.datasets.base import Bunch
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer#feature_extraction‘特征提取’,Convert a collection of raw documents to a matrix of TF-IDF features.

'''
The sklearn.feature_extraction module deals with feature extraction from raw data. 
It currently includes methods to extract features from text and images.
The sklearn.feature_extraction.text submodule gathers utilities to build feature vectors from text documents.
'''

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB #朴素贝叶斯分类算法，Naive Bayes classifier for multinomial models

'''
The multinomial Naive Bayes classifier is suitable for classification with discrete features
 (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts.
  However, in practice, fractional counts such as tf-idf may also work.
'''

from sklearn.metrics import classification_report#metrics度量，分类报告Build a text report showing the main classification metrics

## classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2)   
Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) target values.很明显，这是真实的目标值（样本数，目标特征值）
    y_pred : 1d array-like, or label indicator array / sparse matrix。这是预测对象
    ------------- 
sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。 
主要参数:     
y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。     
y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。     
labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。     
target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。     
sample_weight：类似于shape = [n_samples]的数组，可选项，样本权重。     
digits：int，输出浮点值的位数．

原文：https://blog.csdn.net/akadiao/article/details/78788864 


[参考代码](https://github.com/RiversLEECN/Machine_learning_practice/blob/master/Xin_Wen_Fen_Lei.py)

In [44]:
#text_train_cnews='data/char-level/cnews.train.txt'
#text_val_cnews='data/char-level/cnews.val.txt'
#text_test_cnews='./data/char-level/cnews.test.txt'

text_category_cnews='./data/char-level/cnews.category.txt'
stop_word_file ='./data/中文停用词库.txt'

text_jieba_cnews ='./data/word_level/cnews.jieba.txt'

In [38]:
def get_stop_words(filename=stop_word_file):
    stop_word=[]
    for line in open(filename,'r',encoding='utf-8-sig'):
        stop_word.append(line.strip())
    return stop_word

stop_word = get_stop_words(stop_word_file)
print(stop_word[:10])

[',', '?', '、', '。', '“', '”', '《', '》', '！', '，']


## 填坑系列  
读取文件过程中发现一个问题：已有记事本文件（非空），转码 UTF-8，复制到pycharm中，在开始位置打印结果会出现  \ufeff, 打印代码如下  
```
category={}#将类别名和类别数值存储在字典里面

for line in open(text_category_cnews,'r',encoding='utf-8'):

    c,label=line.strip('\r\n').split('\t')#体育 0；财经 1；房产 2；
    category[c]=label

    #根据类别名获取类别的数值
print(category)
```   
{'\ufeff体育': '0', '财经': '1', '房产': '2', '家居': '3', '教育': '4', '科技': '5', '时尚': '6', '时政': '7', '游戏': '8', '娱乐': '9'}   
**只需改一下编码就行，把 UTF-8 编码 改成 UTF-8-sig**   

[去除\ufeff](https://www.cnblogs.com/chongzi1990/p/8694883.html)

In [37]:
class Categories:
    def __init__(self,filename):#初始化
        self.category={}#将类别名和类别数值存储在字典里面
        for line in open(filename,'r',encoding='utf-8-sig'):
            
            c,label=line.strip('\r\n').split('\t')#体育 0；财经 1；房产 2；
            self.category[c]=label

    #根据类别名获取类别的数值
    def get_category_label(self,name):
        return self.category[name]

categories=Categories(text_category_cnews)

#h=categories.get_category_label('体育')
#print(h)

0


In [39]:
with open('./data/char-level/cnews.test.txt','r',encoding='utf-8') as f:
    lines=f.readlines()#一开始写成了readline，然后找了半天都没有找出来哪里报错了，读取整篇文档
    labels = []
    contents = []
    for line in lines:
        label,content=line.strip('\r\n').split('\t')
        labels.append(label)
        content_list=list(jieba.cut(content))#jieba分词
        content_word=''

        #将list里面的元素（词），用‘’拼接成字符串
        for word in content_list:
            word=word.strip()
            if word !='' and word not in stop_word:#不等于空的数值，进行存储
                content_word +=word+' '#使用空格进行区分
        contents.append(content_word)


In [40]:
print(len(contents))
print(labels[10])
print(contents[10])
print(set(labels))#打印出labels中不重复的键值

10000
体育
数据 揭秘 巨头 关键球 能力 韦德 效率 难比 皇帝 图 新浪 体育讯 北京 时间 月 29 日 迈阿密 时间 月 28 日 消息 热火 巨头 中 最好 比赛 终结者 关键时刻 进攻 效率 最高 问题 永远 出现 一个 标准答案 数据 却 提供 一些 判断 标准 著名 篮球 数据 网站 82games com 标准 比赛 最后 五分钟 双方 分差 分 之内 五分钟 称之为 关键时刻 本赛季 热火 所有 关键时刻 中 热火 巨头 效率 下 表格 得知 见表一 发现 韦德 本赛季 关键时刻 数据 远 以往 赛季 出色 整体 投篮 表现 实际上 詹姆斯 波什都 差 投篮 命中率 低于 四成 四成 以上 一项 有趣 数据 表格 中 使用 值 USG% 指 球员 关键时刻 获得 球权 次数 占 全队 比重 詹姆斯 韦德 分享 热火 关键时刻 大部分 球权 得球率 高达 34.8% 29.6% 波什 不到 一半 仅为 15.5% 关键时刻 范围 缩小 定义 比赛 最后 一分钟 分差 分 之内 比赛 时间 超级 关键时刻 巨头 表现 见表二 看到 表一 相似 詹姆斯 韦德 表现 可谓 差强人意 整体 表现 依然 强 后者 波什 一共 获得 次 出手 机会 无一 命中 有趣 超级 关键时刻 热火 主帅 似乎 信任 詹姆斯 使用 值 USG% 得球率 高 达到 惊人 46.0% 热火 超级 关键时刻 球权 交给 皇帝 完成 詹姆斯 超级 关键时刻 投篮 命中率 远 低于 平均水平 仅为 37.5% 正是 人 诟病 一个 合格 绝杀 真正 原因 所在 再进一步 缩小 范围 比赛 最后 24 秒 分差 分 之内 比赛 时间 生死 时刻 实际上 绝杀 机会 巨头 有着 发挥 见表三 波什 一共 只 获得 两次 绝杀 机会 打成 样本数 太低 两人 比较 没太多 意义 看看 詹姆斯 韦德 表现 韦德 一共 获得 次 绝杀 机会 唯一 打成 那次 去年 11 月初 主场 战 爵士 终场 前 17 秒 一记 三分 帮 热火 比分 追成 114 平 随后 闪电侠 犯规 对手 两罚 两中 锁定 胜局 次 全部 失手 包括 月初 客场 雄鹿 空位 跳投 失手 一月 中旬 客场 公牛 时 次 生死 时刻 先是 挑篮 失手 压哨 三分 失手 命中 追平 比分 错失 月初 主场 公牛 生

## 填坑系列   
Python写循环程序的时候遇到 TypeError: ‘int’ object is not iterable，原因是循环中使用的应该是一组数，将  
for i in len(A)    
改为for i in range(len(A))

In [41]:
with open('./data/word_level/cnews.jieba.txt','a',encoding='utf-8') as f1:#用w的方式，每读取一个文件，输出的文本会将之前的数据覆盖了，用a的话就不会，追加
    for i in range(len(contents)):
        label = labels[i]
        content_word = contents[i]
        wordli='%s\t%s\n'%(label,content_word.strip())#存储分词后的新闻内容
        f1.write(wordli)

In [45]:
def get_data(input_file_name,category_file):#将输入的文本文件进行分词拆分、然后分类

    categories=Categories(category_file)
    labels = []
    contents = []

    with open(input_file_name,'r',encoding='utf-8') as f:      
        lines=f.readlines()

    for line in lines:
        category,content=line.strip('\r\n').split('\t')#由于wordli又通过\t和\n将分类和对应的新闻又组合在了一起
        contents.append(content)
        label=categories.get_category_label(category)#label的取值为0到9，代表体育、财经、房产。。。。。
        labels.append(label)
    return contents,labels

contents1, labels1 = get_data(text_jieba_cnews,text_category_cnews)
print(contents1[0],labels1[0])

鲍勃 库西 奖归 属 NCAA 最强 控卫 坎巴 弗神 新浪 体育讯 本赛季 NCAA 进入 末段 各项 奖项 评选 出炉 评选 最佳 控卫 鲍勃 库西 奖 下周 最终 四强 战时 公布 鲍勃 库西 奖是 由奈 史密斯 篮球 名人堂 提供 旨在 奖励 年度 最佳 大学 控卫 最终 获奖 球员 以下 几名 热门 人选 中 产生 NCAA 疯狂 三月 专题 主页 上线 点击 链接 查看 精彩内容 吉梅尔 弗雷 戴特 杨百翰 大学 弗神 吉梅尔 弗雷 戴特 一直 备受 关注 一名 射手 他会用 终结 对手 脚踝 变向 掉 面前 防守 任意 一支 手 完成 得分 犯规 提前 两份 划入 帐 下 一名 命中率 高达 90% 罚球 手 弗雷 戴特 具有 所有 伟大 控卫 具备 一点 特质 一位 赢家 一位 领导者 整个 赛季 至始 至终 稳定 领导 球队 前进 这是 无可比拟 杨百翰 大学 主教练 戴夫 罗斯 称赞 道 得分 能力 毋庸置疑 认为 带领 球队 获胜 能力 最 重要 控卫 职责 主场 之外 比赛 客场 中 立场 取胜 19 场 表现 很棒 弗雷 戴特 能否 NBA 取得成功 专业人士 资格 做出 判断 喜爱 凯尔特人 主教练 多克 里 弗斯 说道 很棒 看过 ESPN 片段 剪辑 剪辑 超级 巨星 认为 成为 一名 优秀 NBA 球员 诺兰 史密斯 杜克大学 赛季 球队 宣布 一天 控卫凯瑞 厄尔 文因 脚趾 伤病 缺席 赛季 大部分 比赛 后 诺兰 史密斯 接管 球权 进攻 端上 足 发条 ACC 联盟 杜克大学 所在 分区 得分 榜上 名列前茅 分区 助攻 榜上 占据 头名 众强 林立 ACC 联盟 前无古人 认为 全美 球员 凯瑞 厄尔 文 受伤 后 好 接管 球队 之前 毫无准备 杜克 主教练 迈克 沙舍 夫斯基 赞扬 道 他会 比赛 带入 节奏 得分 组织 领导 球队 无所不能 现在 攻防 俱佳 持球 人 防守 提高 拥有 辉煌 赛季 坎巴 沃克 康涅狄格 大学 坎巴 沃克 带领 康涅狄格 赛季 毛伊岛 邀请赛 一路 力克 密歇根州 肯塔基 等队 夺冠 他场 30 分 助攻 得到 最佳 球员 大东 赛区 锦标赛 全国 锦标赛 中 他场 27.1 分 6.1 篮板 5.1 次 助攻 依旧 给力 疯狂 表现 赛季 疯狂 表现 结束 赛季 全国 锦标赛 中 前

# #  权重策略TF——IDF

* TF-IDF(Term frequency * Inverse Doc Frequency)词权重  

* 在较低的文本语料库中，一些词非常常见（例如，英文中的“the”，“a”，“is”），因此很少带有文档实际内容的有用信息。   

* 如果我们将单纯的计数数据直接喂给分类器，那些频繁出现的词会掩盖那些很少出现但是更有意义的词的频率。  
****
* 为了重新计算特征的计数权重，以便转化为适合分类器使用的浮点值，通常都会进行tf-idf转换。  

* 词重要性度量一般使用文本挖掘的启发式方法：TF-IDF。
 这是一个最初为信息检索（作为搜索引擎结果的排序功能）开发的词加权机制，在文档分类和聚类中也是非常有用的   

* 由于tf-idf经常用于文本特征，因此有另一个类称为TfidfVectorizer，

* 将CountVectorizer和TfidfTransformer的所有选项合并在一个模型中



In [51]:
#tfidf（权重列表）
#vocabulary（词典索引）

#使用特征提取函数TfidfVectorizer初始化向量空间模型

vector=TfidfVectorizer(sublinear_tf=True,max_df=0.5)#提取函数的初始化，啥数据都没有处理。选择能代表新闻特征、独一无二的词汇，词频大于50%的就被过滤掉？？？如果过大、过小会如何？

tfidf=vector.fit_transform(contents1)#contents只有新闻内容，没有分类。用df-idf训练转化，获得if-idf权值矩阵：fit_transform

vocabulary=vector.vocabulary_#词典索引，统计词频


In [54]:
#构建分类器

x_train,x_test,y_train,y_test=train_test_split(tfidf,labels1,test_size=0.2,random_state=100)
'''
X_train格式：

0, 592)	0.05975232195788132

  (0, 59)	0.07286741411594184

  (0, 697)	0.07286741411594184

  (0, 296)	0.07286741411594184

  (0, 224)	0.07286741411594184

  (0, 469)	0.07286741411594184

  (0, 513)	0.07286741411594184

  (0, 514)	0.07286741411594184

  (0, 414)	0.07286741411594184

  (0, 517)	0.07286741411594184

  (0, 84)	0.07286741411594184

  Y_train格式：
  ['0', '0', '0', '0']

'''
#以上，提取新闻文档中的词频和对应的新闻分类代号
"""
random_state : int, RandomState instance or None, optional (default=None)If int, 

random_state is the seed used by the random number generator; If RandomState instance, 

random_state is the random number generator; 

If None, the random number generator is the RandomState instance used by np.random.
将总样本划分为100份，从每份中取20%作为训练集，这样的话，可以使得训练集和测试集的误差最小最小。
"""

'\nrandom_state : int, RandomState instance or None, optional (default=None)If int, \n\nrandom_state is the seed used by the random number generator; If RandomState instance, \n\nrandom_state is the random number generator; \n\nIf None, the random number generator is the RandomState instance used by np.random.\n将总样本划分为100份，从每份中取20%作为训练集，这样的话，可以使得训练集和测试集的误差最小最小。\n'

In [55]:
#先实例化模型，然后调用methods，如调用fit、predict

nb=MultinomialNB(alpha=0.01)#实例化模型 alpha: Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).

nb.fit(x_train,y_train)#训练模型	Fit Naive Bayes classifier according to X, y

y_pred=nb.predict(x_test)#预测测试集X; Perform classification(分类) on an array of test vectors X.

print(classification_report(y_test,y_pred))#打印输出评分

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       214
           1       0.98      0.99      0.99       203
           2       0.91      0.89      0.90       196
           3       0.91      0.93      0.92       196
           4       0.98      0.97      0.97       193
           5       1.00      0.99      0.99       195
           6       0.97      0.96      0.96       226
           7       0.96      0.95      0.96       196
           8       0.99      0.98      0.99       192
           9       0.93      0.98      0.96       189

   micro avg       0.96      0.96      0.96      2000
   macro avg       0.96      0.96      0.96      2000
weighted avg       0.96      0.96      0.96      2000



In [56]:
nb.score(x_test, y_test)

0.96350000000000002